# Jupyter

> Obsidian App Jupyter Notebook Rendering support   `

In [ ]:
#| default_exp jupyter

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from memexplatform_obsidian.settings import ObsidianConfig
from enum import Enum
import urllib
from fasthtml.common import *
from monsterui.all import *
from fasthtml.jupyter import *
from collections.abc import Callable
import json
import pathlib
from pathlib import Path
from memexplatform_obsidian.commons import Page
from datetime import datetime, timezone

In [ ]:
# p = pathlib.Path(".")
# p.read_json?

In [ ]:
#| export
def extract_cells(nb):
    cells = []
    for cell in nb.get("cells", []):
        ctype = cell.get("cell_type")
        source = "".join(cell.get("source", []))
        outputs = []
        if ctype == "code":
            # extract outputs already in the notebook
            for out in cell.get("outputs", []):
                # handle different types
                if out.get("output_type") == "stream":
                    outputs.append({"type": "stream", "text": "".join(out.get("text", []))})
                elif out.get("output_type") == "execute_result":
                    data = out.get("data", {})
                    # text/plain may be present
                    text = data.get("text/plain", "")
                    outputs.append({"type": "execute_result", "text": text})
                elif out.get("output_type") == "error":
                    tb = out.get("traceback", [])
                    outputs.append({"type": "error", "text": "\n".join(tb)})
        cells.append({"cell_type": ctype, "source": source, "outputs": outputs})
    return cells

In [ ]:
#| export
_counter = 0
def _gen_id(prefix="f"):
    global _counter
    _counter += 1
    return f"{prefix}{_counter}"

def create_flippable_card(content, source_code, outputs=None, extra_cls=None):
    """
    Create a card that shows `content` (front), `source_code` toggled on "See Source",
    and optionally `outputs` toggled on "See Output".
    """
    uid = _gen_id("card")

    # Button to toggle source code
    btn_source = Button(
        "See Source",
        cls=ButtonT.primary,
        **{"data_uk_toggle": f"target: #{uid}-source"}
    )
    # Hidden Div for source
    source_div = Div(
        Pre(Code(source_code, classes="hljs language-python")),
        id=f"{uid}-source",
        hidden=True,
        cls="mt-2 p-2 bg-gray-100 rounded"
    )

    children = [btn_source, content, source_div]

    if outputs:
        btn_outputs = Button(
            "See Output",
            cls=ButtonT.secondary,
            **{"data_uk_toggle": f"target: #{uid}-output"}
        )
        output_div = Div(
            *[Pre(o) for o in outputs],
            id=f"{uid}-output",
            hidden=True,
            cls="mt-2 p-2 bg-gray-100 rounded text-gray-800"
        )
        children.insert(1, btn_outputs)  # place output button before content maybe
        children.append(output_div)

    card = Card(*children, cls=("my-6 " + extra_cls) if extra_cls else "my-6")
    return card

In [ ]:
#| export
class JupyterPage(Page):

    def __init__(self, text, fpath=None, created_time=None, modified_time=None, file_size=None):
        if isinstance(text, str):
            try:
                text = json.loads(text)
            except Exception:
                # fallback: treat it as error message
                text = {"cells": [{"cell_type": "markdown", "source": text}]}
        super().__init__(text, fpath, created_time, modified_time, file_size)

    
    @classmethod
    def from_file_path(cls, file_path: Path):
        text: str = None
        file_path = pathlib.Path(file_path)
        if not file_path.is_file():
            # Handle cases where the path does not point to a regular file
            if not file_path.exists():
                text = f"[Error: File not found at path: {file_path}]"
            else:  # Path exists but is not a file (e.g., a directory, or a special file)
                text = f"[Error: Path exists but is not a regular file: {file_path}]"
        else:
            try:
                text = file_path.read_json(encoding='utf-8')

            except:
                pass
            s = file_path.stat(); s.st_size
            created_time = datetime.fromtimestamp(s.st_ctime_ns / 1e9, tz=timezone.utc)
            # mtime (last modification time in ns)
            modified_time = datetime.fromtimestamp(s.st_mtime_ns / 1e9, tz=timezone.utc)
            file_size = s.st_size
        return cls(text=text, 
                   fpath=file_path, 
                   created_time=created_time, 
                   modified_time=modified_time, 
                   file_size=file_size)
    @property
    def ast(self):
        return self._text

    @property
    def title(self):
        return self._fpath.stem

    @property
    def links(self):
        return []

    @property
    def html(self):
        cells = extract_cells(self._text)
        blocks = []
        for cell in cells:
            if cell["cell_type"] == "markdown":
                if cell["source"].strip():
                    md_html = render_md(cell["source"])
                    blocks.append(Card(
                        Safe(md_html),
                        cls="my-4 shadow"
                    ))
            elif cell["cell_type"] == "code":
                if cell["source"].strip():
                    # content: show code front
                    content = Pre(Code(cell["source"], classes="hljs language-python"))
                    # prepare outputs list of plain strings
                    outs = [o["text"] for o in cell.get("outputs", []) if o.get("text")]
                    blocks.append(
                        create_flippable_card(
                            content=content,
                            source_code=cell["source"],
                            outputs=outs,           # now allowed
                            extra_cls="border border-gray-300"
                        )
                    )
        ft = Container(*blocks, cls="p-6 space-y-6")
        return to_xml(ft)

In [ ]:
#| notest
path = '/Users/rahul1.saraf/rahuketu/programming/notesobs/pages/ThinkBayes.01_Probability.ipynb'
# nb_content = json.loads(Path(path).read_text()); nb_content
page = JupyterPage.from_file_path(path); page.html

'<div class="uk-container p-6 space-y-6">\n  <div class="uk-card my-4 shadow">\n    <div class="uk-card-body space-y-6"><p class="text-lg leading-relaxed mb-6">Probability is defined as a fraction of a finite set. The word "random" in "random probability" means everyone has the same chance of being chosen, but the chance is probably close to 2%.</p>\n</div>\n  </div>\n  <div class="uk-card my-6 border border-gray-300">\n    <div class="uk-card-body space-y-6">\n<button data-uk-toggle="target: #card1-source" type="submit" class="uk-btn uk-btn-primary">See Source</button><pre><code classes="hljs language-python">import pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport random</code></pre>      <div hidden id="card1-source" class="mt-2 p-2 bg-gray-100 rounded">\n<pre><code classes="hljs language-python">import pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport random</code></pre>      </div>\n    </div>\n  </div>\n  <div class="uk-card my-6 bo

In [ ]:
# render_nb(path)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()